# Short Circuit 1: Season 1 Election Notes

>**FORBIDDEN KNOWLEDGE WARNING**
>
>This document contains some information that is not currently present on the main site and may be considered spoilers.
>Continue at your own discretion. Some information in this document cannot be shared on the Discord without spoiler tags.


In [1]:
import pandas
%matplotlib inline
from blaseball_mike.models import *
from blaseball_mike.tables import StatType
import plotly.express as plot
import plotly.io as _pio
import plotly.subplots as subplot
from IPython.display import display, Markdown
from copy import deepcopy

import os
import sys
pdir = os.path.abspath(os.path.join(os.path.dirname(''), os.path.pardir))
sys.path.append(pdir)
from display import *
from blessings import *
sys.path.remove(pdir)

_pio.renderers.default = "notebook_connected"

league = League.load()
pies = [x for x in league.teams.values() if x.full_name == "Philly Pies"][0]

sim = SimulationData.load()
display(Markdown(f"**Last Updated Season {sim.season}, Day {sim.day}**"))

**Last Updated Season 1, Day 0**

---
## Amplifications

Amplification are chosen by majority votes for each team. Each team will receive `1` amplification.


**Your Team's Lineup will receive a Small Batting Boost.**

**Your Team's Rotation will receive a Small Pitching Boost.**

**Your Team's Lineup will receive a Large Baserunning Boost.**

**Your Team's Lineup will receive a Large Defense Boost.**

All the Amplifications will boost team-wide stats in one area. It is unknown what the difference between "Small" and
"Large" is.


___
## Distortions

Distortions are selected by raffle across all teams. Each distortion will be awarded to one random team, where teams with
more votes are more likely to be selected. Teams may only receive at most 1 distortion.

### Fire Blood

**Give a random Player in your Team's Rotation Fire Blood.**

Fire Blood grants the "Fiery" Modification which has the description "This Player has a chance of throwing Double
Strikes."

### Electric Blood

**Give a random Player in your Team's Rotation Electric Blood.**

Electric Blood grants the "Electric" Modification which has the description "When this Player is Pitching, they'll have
a chance of zapping away Balls."

### Strike Blood

**Give a random Player in your Team's Lineup Strike Blood.**

Strike is one of the 2 new bloods, so the effect is unknown.

### Ball Blood

**Give a random Player in your Team's Lineup Ball Blood.**

Ball is one of the 2 new bloods, so the effect is unknown.

### Arm Cannon

**Give the Arm Cannon to a random Player in your Team's Rotation.**

This would award a random pitcher with the Arm Cannon Item.


### Gunblade Bat

**Give the Gunblade Bat to a random Player in your Team's Lineup.**

This would award a random batter with the Gunblade Bat Item.

### Rocket Shoes

**Give Rocket Shoes to a random Player in your Team's Lineup.**

This would award a random batter with the Rocket Shoes Item.

### Catcher's Mitt

**Give Catcher's Mitt to a random Player in your Team's Lineup.**

This would award a random batter with the Catcher's Mitt Item.

### Lead Investment

**Move the Best Batter in your Team's Lineup to first in the Lineup.**

See below for analysis.

In [2]:
temp_pies = pies.lineup.copy()
temp_pies.sort(key=lambda x: x.get_hitting_rating(), reverse=True)
best_bat = temp_pies[0]
display(f"Move {best_bat.name} to the first slot in the lineup.")
rest_of_them = [x for x in pies.lineup if x.id != best_bat.id]
display(set_heatmap(get_batting_stlats([best_bat] + rest_of_them), maxVal=1.0))

'Move Worf Hatchler to the first slot in the lineup.'

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Worf Hatchler,0.718041,0.950081,0.672136,0.591004,0.463866,0.584336,0.256903,0.457766,0.346630
Magma Projector,0.459058,0.851396,0.261210,0.348121,0.393707,0.464666,0.543143,0.997501,0.964074
Lance Anthemum,0.040804,0.000475,0.482953,0.558319,0.218143,0.869002,0.846691,0.262651,0.757512
Maria Lin,0.453856,0.503086,0.390041,0.863518,0.601501,0.861243,0.037214,0.467341,0.449169
Mugs Dropper,0.273535,0.309323,0.112244,0.653758,0.659843,0.068277,0.083171,0.318857,0.488464
Chrys McGhee,0.536418,0.280475,0.877597,0.538791,0.602464,0.571543,0.918841,0.312086,0.527615
Eva Roche,0.558674,0.479421,0.643668,0.383185,0.431371,0.313640,0.604652,0.493201,0.011742
Buck Yuniesky,0.187877,0.167948,0.138737,0.373232,0.083748,0.323448,0.023790,0.181509,0.777762
Whiskers Falconer,0.493468,0.695906,0.423098,0.866845,0.050597,0.419942,0.875366,0.752295,0.377555


### Default Swap

**Swap the Worst Batter in your Team's Lineup with the Worst Pitcher in your Team's Rotation.**

See below for analysis.

In [3]:
worst_batter = sort_lineup(pies, num=1)[0]
worst_pitcher = sort_rotation(pies, num=1)[0]
display(f"Swap {worst_batter.name} to rotation and {worst_pitcher.name} to lineup.")
display(pandas.DataFrame(swap_players(pies, worst_pitcher, worst_batter)))

display(set_heatmap(get_batting_stlats(worst_pitcher), maxVal=1.0))
display(set_heatmap(get_pitching_stlats(worst_batter), maxVal=1.0))

'Swap Lance Anthemum to rotation and Stacy Heartfield to lineup.'

,Pies
batting_change,+5.3142%
pitching_change,+1.1559%
baserunning_change,+5.3727%
defense_change,+2.3759%


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Stacy Heartfield,0.519083,0.890048,0.284424,0.638109,0.225520,0.002401,0.337685,0.534442,0.696127


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Lance Anthemum,0.373650,0.601195,0.238249,0.533289,0.680156,0.392261,0.096427


### Cold Call

**Call the Alternate for the 2 Worst Pitchers in your Team's Rotation.**

An Alternate is a reroll of the player's stats. It is unknown if it will maintain combined star total similarly to later
Expansion Era Alternates. This should also reroll players if they are still the worst player after their first reroll.
See below for the list of players this would reroll. For reference, the statistical average reroll would have ~1.5
pitching stars (or ~0.3 Pitching Rating).

In [4]:
display(set_heatmap(get_pitching_stlats(sort_rotation(pies, 2)), maxVal=1.0))

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Stacy Heartfield,0.315853,0.509052,0.389210,0.239287,0.181181,0.124524,0.352019
Stamp Chavez,0.364289,0.443009,0.678260,0.339319,0.271439,0.002037,0.551335


### Long Distance Call

**Call the Alternate for the 4 Worst Batters in your Team's Lineup.**

An Alternate is a reroll of the player's stats. It is unknown if it will maintain combined star total similarly to later
Expansion Era Alternates. This should also reroll players if they are still the worst player after their first reroll.
See below for the list of players this would reroll. For reference, the statistical average reroll would have ~2 batting
stars (or ~0.4 Batting Rating).


In [5]:
display(set_heatmap(get_batting_stlats(sort_lineup(pies, 4)), maxVal=1.0))

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Lance Anthemum,0.040804,0.000475,0.482953,0.558319,0.218143,0.869002,0.846691,0.262651,0.757512
Buck Yuniesky,0.187877,0.167948,0.138737,0.373232,0.083748,0.323448,0.023790,0.181509,0.777762
Mugs Dropper,0.273535,0.309323,0.112244,0.653758,0.659843,0.068277,0.083171,0.318857,0.488464
Maria Lin,0.453856,0.503086,0.390041,0.863518,0.601501,0.861243,0.037214,0.467341,0.449169



---
## Appendix
* [Description of Attributes](https://www.blaseball.wiki/w/Player_Attributes)
* [Stlat Viewer](http://yoori.space/astrology/#philly-pies-gamma)
* [Historical Player Graphs](http://yoori.space/hloroscopes/)